<h1 align="center"> Sensordatenfusion Tutorium 04 </h1>

<h3 align="center"> Messmodel und Updater </h3>

Ziel dieses Tutoriums ist es, $x_{k|k}$ und $P_{k|k}$ zu erhalten. 
Dies wird ermöglicht durch eine neue Messung $z_k$. 
Zudem benötigen wir noch die Messmatrix $H_k$ und ein Rauschen $R_k$.

Das Vorgehen in der Filterung ist nun folgendes:

(1) Berechnung der Innovation $v_{k|k-1}$ und ihrer Kovarianz $S_{k|k-1}$ 

  $v_{k|k-1}$ = $z_k$ - $H_k x_{k|k-1}$
  
  $S_{k|k-1}$ = $H_k P_{k|k-1} H^T_k$ + $R_k$
  
(2) Berechnung der Gewichtsmatrix $W_{k|k-1}$  

  $W_{k|k-1}$ = $P_{k|k-1} H^T_k S_{k|k-1}$


(3) Berechnung $x_{k|k}$ und $P_{k|k}$

  $x_{k|k}$ = $x_{k|k-1} + W_{k|k-1} v_{k|k-1}$ 
  
  $P_{k|k}$ = $P_{k|k-1} - W_{k|k-1} S_{k|k-1} W^T_{k|k-1}$

In [23]:
import scipy as sp
import numpy as np
from scipy.stats import multivariate_normal

from stonesoup.base import Property
from stonesoup.types.array import CovarianceMatrix
from stonesoup.models.base import LinearModel, GaussianModel
from stonesoup.models.measurement.base import MeasurementModel

<h3 align="center"> Messmodell erstellen </h3>

In [ ]:
class SDFMessmodell(MeasurementModel, LinearModel, GaussianModel):
    
    @property
    def ndim_meas(self):
        return 2
    
    def matrix(self, **kwargs):
        # model_matrix = np.array([["""Erste Zeile der Messmatrix"""], [""" Zweite Zeile der Messmatrix"""]])
        model_matrix = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
        return model_matrix
    
    def covar(self):
        # return  np.array([["""Erste Zeile der Rauschmatrix R"""], ["""Zweite Zeile der Rauchmatrix R"""]])
        return np.array([[0.75, 0],[0, 0.75]])
    
    def rvs(self):
        # sample ziehen aus der Kovarianzmatrix
        noise = multivariate_normal.rvs(sp.zeros(self.ndim_meas), self.covar(), 1)
        return noise.reshape((-1, 1))
    
    def pdf(self):
        pass
    
    
    

In [25]:
messmodell = SDFMessmodell(4, (0, 2))

In [26]:
messmodell.matrix()

array([[1, 0, 0, 0],
       [0, 0, 1, 0]])

In [27]:
messmodell.covar()

array([[0.75, 0.  ],
       [0.  , 0.75]])

In [28]:
print(messmodell.rvs())

[[-0.49726351]
 [ 0.33421274]]


<h3 align="center"> Imports für den Updater </h3>

In [29]:
from functools import lru_cache

from stonesoup.updater.base import Updater
from stonesoup.base import Property
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.prediction import GaussianMeasurementPrediction
from stonesoup.types.update import GaussianStateUpdate

<h3 align="center"> Updater erstellen </h3>

In [ ]:
class SDFUpdater(Updater):
    messprediction = None  # erwartete Messung
    S = None  # erwartete Messkovarianz
    Pxy = None  # 

    @lru_cache()
    def get_measurement_prediction(self, state_prediction, measurement_model=SDFMessmodell(4, (0, 2), np.array([[0.75, 0], [0, 0.75]])), **kwargs):
        measurement_matrix = measurement_model.matrix()
        measurement_noise_covar = measurement_model.covar()
        state_prediction_mean = state_prediction.mean
        state_prediction_covar = state_prediction.covar

        self.messprediction = measurement_matrix @ state_prediction_mean
        self.S = measurement_matrix @ state_prediction_covar @ measurement_matrix.T + measurement_noise_covar
        self.Pxy = state_prediction_covar @ measurement_matrix.T

        return GaussianMeasurementPrediction(self.messprediction, self.S,
                                             state_prediction.timestamp,
                                             self.Pxy)

    def update(self, hypothesis, measurementmodel, **kwargs):
        test = self.get_measurement_prediction(hypothesis.prediction, measurementmodel)     # damit messprediction, kamalngain etc berechnet werden
        W = self.Pxy @ np.linalg.pinv(self.S)
        x_post = hypothesis.prediction.mean + W @ (hypothesis.measurement.state_vector - self.messprediction)
        P_post = hypothesis.prediction.covar - (W @ self.S @ W.T)  # Dimensionen passen nicht
        # P_post = (P_post + P_post.T) / 2

        posterior_mean = x_post
        posterior_covar = P_post
        meas_pred_mean = self.messprediction
        meas_pred_covar = self.S
        cross_covar = self.Pxy
        _ = W

        # Augment hypothesis with measurement prediction
        hypothesis = SingleHypothesis(hypothesis.prediction,
                                      hypothesis.measurement,
                                      GaussianMeasurementPrediction(
                                          meas_pred_mean, meas_pred_covar,
                                          hypothesis.prediction.timestamp,
                                          cross_covar)
                                      )

        return GaussianStateUpdate(posterior_mean,
                                   posterior_covar,
                                   hypothesis,
                                   hypothesis.measurement.timestamp)